In [114]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import zscore
from collections import Counter
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
df =  pd.read_csv("movies.csv")

In [116]:
display(df.columns)

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations'],
      dtype='object')

In [117]:
df = df.drop_duplicates(subset=['title', 'release_date'])

In [118]:
idioma_para_continente = {
    # América do Norte
    'en': 'America_do_Norte',

    # Ásia
    'ko': 'Asia',
    'cn': 'Asia', # Chinês (Simplificado/Mandarin)
    'hi': 'Asia', # Hindi
    'ja': 'Asia', # Japonês
    'zh': 'Asia', # Chinês (Genérico/Variedades)
    'te': 'Asia', # Telugu
    'ar': 'Asia', # Árabe (Também África)
    'ml': 'Asia', # Malayalam
    'kn': 'Asia', # Kannada
    'ta': 'Asia', # Tamil
    'km': 'Asia', # Khmer
    'he': 'Asia', # Hebraico
    'bn': 'Asia', # Bengali
    'ms': 'Asia', # Malaio
    'bo': 'Asia', # Tibetano
    'kk': 'Asia', # Cazaque (Também Europa)
    'ka': 'Asia', # Georgiano (Também Europa)
    'ku': 'Asia', # Curdo (Também Europa)
    'ur': 'Asia', # Urdu
    'ps': 'Asia', # Pashto
    'mr': 'Asia', # Marathi
    'lo': 'Asia', # Laosiano
    'ne': 'Asia', # Nepalês


    # Europa
    'es': 'Europa', # Espanhol (Também Américas)
    'no': 'Europa', # Norueguês
    'fi': 'Europa', # Finlandês
    'ru': 'Europa', # Russo (Também Ásia)
    'uk': 'Europa', # Ucraniano
    'nl': 'Europa', # Holandês
    'it': 'Europa', # Italiano
    'fr': 'Europa', # Francês (Também Américas, África)
    'pl': 'Europa', # Polonês
    'pt': 'Europa', # Português (Também Américas, África)
    'de': 'Europa', # Alemão
    'ro': 'Europa', # Romeno
    'sr': 'Europa', # Sérvio
    'sv': 'Europa', # Sueco
    'gl': 'Europa', # Galego
    'tr': 'Europa', # Turco (Também Ásia)
    'cs': 'Europa', # Tcheco
    'el': 'Europa', # Grego
    'ga': 'Europa', # Irlandês
    'mk': 'Europa', # Macedônio
    'sh': 'Europa', # Sérvio-Croata
    'hu': 'Europa', # Húngaro
    'eu': 'Europa', # Basco
    'lv': 'Europa', # Letão
    'sk': 'Europa', # Eslovaco
    'et': 'Europa', # Estoniano
    'hy': 'Europa', # Armênio (Também Ásia)
    'sq': 'Europa', # Albanês
    'se': 'Europa', # Sami Setentrional
    'sl': 'Europa', # Esloveno
    'cy': 'Europa', # Galês
    'lt': 'Europa', # Lituano
    'hr': 'Europa', # Croata
    'bg': 'Europa', # Búlgaro
    'yi': 'Europa', # Iídiche
    'eo': 'Europa', # Esperanto
    'nb': 'Europa', # Bokmål Norueguês


    # África
    'sw': 'Africa', # Suaíli
    'af': 'Africa', # Africâner
    'am': 'Africa', # Amárico
    'zu': 'Africa', # Zulu
    'xh': 'Africa', # Xhosa
    'wo': 'Africa', # Uolofe
    'bm': 'Africa', # Bambara


    # América do Sul
    'qu': 'America_do_Sul', # Quechua
    'ay': 'America_do_Sul', # Aimará


    # Oceania
    'mi': 'Oceania', # Maori


    # Outros / Desconhecido / Códigos Especiais
    'id': 'Asia',   # Indonésio (Geralmente Ásia, mas para simplificar, pode agrupar)
    'is': 'Europa',   # Islandês (Europa, mas pode ser um grupo separado se for muito raro)
    'tl': 'Asia',   # Tagalo (Filipino) (Ásia, mas pode agrupar)
    'dz': 'Asia',   # Dzongkha (Ásia, Butão)
    'xx': 'Outros',   # Sem linguagem / Linguagem desconhecida (Código especial)
    'la': 'Outros',   # Latim (Histórico)
    'vi': 'Asia',   # Vietnamita (Ásia, mas pode agrupar)
    'bs': 'Europa',   # Bósnio (Europa, mas pode agrupar)
    'iu': 'America_do_Norte',   # Inuktitut (América do Norte, Ártico)
    'pa': 'Asia',   # Punjabi (Ásia, mas pode agrupar)
    'th': 'Asia', # Tailandês (Ásia, mas para manter consistência com Ásia acima)

    # Adicionar os idiomas restantes que não foram listados explicitamente acima
    # ...
}

# Supondo que 'df' já existe e 'idioma_para_continente' foi definido

# 1. Mapear 'original_language' para 'continente'
df['continente'] = df['original_language'].map(idioma_para_continente)

# 2. Preencher idiomas desconhecidos com 'Outros'
df['continente'] = df['continente'].fillna('Outros')

# 3. Criar variáveis dummies para a coluna 'continente'
continente_dummies = pd.get_dummies(df['continente'], prefix='regiao')

# 4. Concatenar as dummies no DataFrame original
df = pd.concat([df, continente_dummies], axis=1)

# 5. Remover as colunas originais de idioma e continente textual, pois já estão representadas nas dummies
df = df.drop(columns=['original_language', 'continente'])

# 6. (Opcional) Mostrar as colunas novas criadas
print("Colunas de continente adicionadas:")
print(continente_dummies.columns.tolist())

Colunas de continente adicionadas:
['regiao_Africa', 'regiao_America_do_Norte', 'regiao_America_do_Sul', 'regiao_Asia', 'regiao_Europa', 'regiao_Oceania', 'regiao_Outros']


In [119]:
df = df[df['status'] == 'Released']

In [120]:
# Converter a coluna para datetime (se ainda não estiver)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Criar as colunas release_year e release_month
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

In [121]:
df = df[
    (df['budget'] > 0) &
    (df['runtime'] > 70) &
    (df['vote_count'] > 30)
]

In [122]:
# Substituir zeros por 1
df['budget'] = df['budget'].replace(0, 1)
df['revenue'] = df['revenue'].replace(0, 1)

# Aplicar transformação logarítmica e criar colunas novas
df['budget_log'] = np.log(df['budget'])
df['revenue_log'] = np.log(df['revenue'])

In [123]:
# --- Separar os gêneros por separador (supondo que seja '-') ---
df['genres'] = df['genres'].fillna('')
df['genres'] = df['genres'].apply(lambda x: x.split('-') if x else [])

# --- Contar todos os gêneros ---
contagem = Counter([genero for sublist in df['genres'] for genero in sublist])

# --- Identificar os gêneros com menos de 600 ocorrências ---
generos_raros = [g for g, count in contagem.items() if count < 600]

# --- Substituir gêneros raros por 'Outros' ---
def substituir_raros(lista):
    return [g if g not in generos_raros else 'Outros' for g in lista]

df['genres'] = df['genres'].apply(substituir_raros)

# --- Recontar os gêneros agora atualizados ---
contagem_atualizada = Counter([genero for sublist in df['genres'] for genero in sublist])
genero_series = pd.Series(contagem_atualizada).sort_values(ascending=False)

# --- Exibir contagem final por gênero ---
display(genero_series)
print(f"\nTotal de gêneros únicos (após agrupar os raros): {len(genero_series)}")

Drama              4923
Comedy             3658
Thriller           2855
Action             2765
Outros             2259
Romance            1819
Adventure          1764
Horror             1676
Crime              1654
Science Fiction    1283
Fantasy            1043
Family              985
Mystery             927
dtype: int64


Total de gêneros únicos (após agrupar os raros): 13


In [124]:
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# 3. Tratar gêneros: garantir que seja lista
df['genres'] = df['genres'].fillna('')
df['genres'] = df['genres'].apply(lambda x: x.split('-') if isinstance(x, str) else x)

# Recontar os gêneros e agrupar os que aparecem menos de 900 vezes como 'Outros'
contagem = Counter([genero for sublist in df['genres'] for genero in sublist])
generos_raros = [g for g, count in contagem.items() if count < 900]

def substituir_raros(lista):
    return [g if g not in generos_raros else 'Outros' for g in lista]

df['genres'] = df['genres'].apply(substituir_raros)

# 4. Transformar em dummies
mlb = MultiLabelBinarizer()
genres_dummies = mlb.fit_transform(df['genres'])
df_genres = pd.DataFrame(genres_dummies, columns=mlb.classes_, index=df.index)

# 5. Concatenar com as colunas numéricas
num_cols = ['popularity', 'runtime', 'vote_average', 'vote_count', 'release_year', 'release_month', 'budget_log', 'revenue_log']
df = pd.concat([df[num_cols], df_genres, continente_dummies], axis=1)

# 6. Remover NaNs
df = df.dropna()

# Exibir as colunas finais
display(df.columns)


Index(['popularity', 'runtime', 'vote_average', 'vote_count', 'release_year',
       'release_month', 'budget_log', 'revenue_log', 'Action', 'Adventure',
       'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'Horror', 'Mystery',
       'Outros', 'Romance', 'Science Fiction', 'Thriller', 'regiao_Africa',
       'regiao_America_do_Norte', 'regiao_America_do_Sul', 'regiao_Asia',
       'regiao_Europa', 'regiao_Oceania', 'regiao_Outros'],
      dtype='object')

In [125]:
# Lista das colunas numéricas
num_cols = ['popularity', 'runtime', 'vote_average', 'vote_count', 'release_year', 'release_month', 'budget_log', 'revenue_log']

# Aplicar z-score apenas nessas colunas
df[num_cols] = df[num_cols].apply(zscore)

# Verificação rápida
display(df[num_cols].describe())

,popularity,runtime,vote_average,vote_count,release_year,release_month,budget_log,revenue_log
count,1.082100e+04,1.082100e+04,1.082100e+04,1.082100e+04,1.082100e+04,1.082100e+04,1.082100e+04,1.082100e+04
mean,-4.202452e-17,-2.941717e-16,-7.144169e-16,-4.202452e-17,-1.239723e-15,-1.155674e-16,-5.883433e-16,-1.680981e-16
std,1.000046e+00,1.000046e+00,1.000046e+00,1.000046e+00,1.000046e+00,1.000046e+00,1.000046e+00,1.000046e+00
min,-1.668467e-01,-1.667319e+00,-5.005979e+00,-4.900498e-01,-4.930496e+00,-1.726311e+00,-7.799781e+00,-1.590577e+00
25%,-1.243436e-01,-6.742983e-01,-6.086279e-01,-4.628570e-01,-3.751450e-01,-8.497626e-01,-4.518754e-01,-1.590577e+00
50%,-9.911378e-02,-2.229252e-01,5.097479e-02,-3.690770e-01,3.081576e-01,2.678544e-02,1.412993e-01,5.024173e-01
75%,-4.631798e-02,4.541345e-01,7.105774e-01,-2.254426e-02,7.067508e-01,9.033335e-01,6.306556e-01,7.255783e-01
max,4.869477e+01,2.257142e+01,2.909253e+00,1.165115e+01,1.276170e+00,1.487699e+00,2.027598e+00,1.253732e+00


In [126]:
# 1. Separar numéricas e dummies
num_cols = ['popularity', 'runtime', 'vote_average', 'vote_count', 'release_year', 'release_month', 'budget_log', 'revenue_log']
dummy_cols = [col for col in df.columns if col not in num_cols]

# 3. Montar array para PCA (já com dummies 0/1)
X = df[num_cols + dummy_cols].values

# 4. Aplicar PCA, mantendo por exemplo 95% da variância
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)

print(f"Número de componentes que mantêm 95% da variância: {pca.n_components_}")

Número de componentes que mantêm 95% da variância: 16
